In [1]:
import numpy as np
import pandas as pd
import os


In [33]:
# Get the current working directory
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")

# List files in the current directory
files_in_directory = os.listdir(current_directory)
print(f"Files in directory: {files_in_directory}")

# Construct the full file path
# Replace 'actual_file_name.csv' with the actual name and extension of your file
file_path = os.path.join(current_directory, 'working file_mahendra.csv')
# or provide the absolute path like: file_path = '/path/to/your/file/actual_file_name.csv'

# Read the CSV file using the file path
df_print = pd.read_csv(file_path)

Current working directory: /content
Files in directory: ['.config', 'Hdbscan_clusternames.csv', 'working file_mahendra.csv', 'hdbscan_cluster.csv', 'predicted labels.csv', 'clustered_job_names.csv', 'sample_data']


<ipython-input-33-b30a53b4ddcb>:15: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_print = pd.read_csv(file_path)


In [3]:
df_print.shape

(1048575, 11)

In [4]:
df_clean = df_print[df_print['JobName'].notna() & (df_print['JobName'].str.strip() != '')]

In [32]:
df_clean.shape

(632958, 11)

In [6]:
df_sample = df_clean.sample(n=10000, random_state=42)
job_names_sample = df_sample['JobName'].astype(str).tolist()


In [7]:
#df_sample = df_sample.drop_duplicates(subset='JobName')

In [8]:
df_sample.shape

(10000, 11)

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(job_names_sample, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [10]:
import numpy as np

df_emb = pd.DataFrame(embeddings)
df_emb['job_name'] = job_names_sample  # keep job name attached


In [11]:
df_emb.shape

(10000, 385)

In [12]:
from sklearn.cluster import KMeans

X = df_emb.drop(columns='job_name').values
kmeans = KMeans(n_clusters=20, random_state=42)
labels = kmeans.fit_predict(X)

df_emb['cluster'] = labels


In [13]:
df_emb.columns

Index([         0,          1,          2,          3,          4,          5,
                6,          7,          8,          9,
       ...
              376,        377,        378,        379,        380,        381,
              382,        383, 'job_name',  'cluster'],
      dtype='object', length=386)

In [34]:
df_emb.shape

(10000, 386)

In [14]:
df_sample['cluster'] = df_emb['cluster'].values


In [15]:
df_sample.to_csv("clustered_job_names.csv", index=False)

In [16]:
# trying the clustering process with HDBSCAN

import hdbscan

# Safely drop non-embedding columns (job_name, KMeans cluster)
X_hdb = df_emb.drop(columns=['job_name', 'cluster']).values

# Run HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=30, metric='euclidean')
hdb_labels = clusterer.fit_predict(X_hdb)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [35]:
df_sample['hdbscan_cluster'] = hdb_labels


In [37]:
df_sample.columns

Index(['DeviceName', 'TrackingPageCount', 'UserDescription', 'JobName',
       'JobDateTime', 'Serial Number', 'Site', 'Site code', 'Model', 'Country',
       'Month', 'cluster', 'hdbscan_cluster'],
      dtype='object')

In [19]:
df_sample.to_csv("hdbscan_cluster.csv", index=False)

In [21]:
df_clustnames = pd.read_csv("Hdbscan_clusternames.csv")

In [22]:
df_clustnames.columns

Index(['DeviceName', 'TrackingPageCount', 'UserDescription', 'JobName',
       'JobDateTime', 'Serial Number', 'Site', 'Site code', 'Model', 'Country',
       'Month', ' K means cluster', 'hdbscan_cluster', 'Cluster name'],
      dtype='object')

In [24]:
# Step 1: Drop rows where cluster or name is missing
df_valid = df_clustnames[['hdbscan_cluster', 'Cluster name']].dropna()

# Step 2: Drop duplicate cluster-name pairs (in case of redundancy)
df_unique = df_valid.drop_duplicates(subset=['hdbscan_cluster', 'Cluster name'])

# Step 3: Create dictionary: {cluster_id: cluster_label}
cluster_name_map = dict(zip(df_unique['hdbscan_cluster'], df_unique['Cluster name']))


In [40]:
print(cluster_name_map)


{7: 'Unattended print', 4: 'Email Job', -1: 'Cannot be grouped', 26: 'Microsoft word document', 22: 'pdf documents', 12: 'Microsoft Outlook Memos', 17: 'P report cluster', 33: 'FRM 8 series pdf', 18: 'P report cluster', 24: 'FRM excel report', 11: 'Reconciliation reports pdf', 31: 'Microsoft Word  FRM document', 16: 'ASUN excel reports', 28: 'Microsoft word document ZINBPR ', 6: 'Site working project QC analysis', 29: 'Microsoft Word P34', 15: 'MBR batch pdf', 20: 'Date cluster jobs and snagx', 23: 'FRM report', 30: 'Microsoft Word - NUS', 13: 'Global Standard pdf', 3: 'Email Job', 19: 'Document cluster', 14: 'Fedex labels pdf', 25: 'FRM 8  and 7 series reports', 9: 'system-generated or developer-defined Report', 0: 'Remote Desktop Printer document', 10: 'system-generated or developer-defined Report', 5: 'Crystal Reports', 21: 'Solids GP excel report', 27: 'Microsoft word ZININSP document', 32: 'FRM 8 series pdf', 8: 'Unattended print', 1: 'Chrome extension html files', 2: 'Chrome exte

In [26]:
# Step 1A: Filter records that already have HDBSCAN clusters and cluster names
df_train = df_clustnames[df_clustnames['Cluster name'].notna()].copy()

# Step 1B: Identify the remaining records (i.e., the "unlabeled" set)
# Assuming full dataset is stored in df_all
df_unlabeled = df_clean[~df_clean['JobName'].isin(df_train['JobName'])].copy()

print("Training records:", df_train.shape)
print("Unlabeled records to predict:", df_unlabeled.shape)


Training records: (10000, 14)
Unlabeled records to predict: (360200, 11)


In [27]:
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder

# Step 2A: Embed job names in training data
model = SentenceTransformer('all-MiniLM-L6-v2')
X_train = model.encode(df_train['JobName'].astype(str).tolist(), show_progress_bar=True)

# Step 2B: Prepare labels
le = LabelEncoder()
y_train = le.fit_transform(df_train['Cluster name'])


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [28]:
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:44:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [29]:
# Step 4A: Embed job names in the unlabeled dataset
X_test = model.encode(df_unlabeled['JobName'].astype(str).tolist(), show_progress_bar=True)

# Step 4B: Predict cluster IDs
y_pred = xgb.predict(X_test)

# Step 4C: Convert back to human-readable cluster names
df_unlabeled['Predicted Cluster name'] = le.inverse_transform(y_pred)


Batches:   0%|          | 0/11257 [00:00<?, ?it/s]

In [30]:
df_unlabeled.head()

,DeviceName,TrackingPageCount,UserDescription,JobName,JobDateTime,Serial Number,Site,Site code,Model,Country,Month,Predicted Cluster name
0,PHITTA-PM0031,1476.0,Torre Annunziata,LIBERAZIONE-DAL-MALIGNO-Preghiere (1).pdf,27-Jan-25,CZBBS650FQ,Pharma_01,ITTA,HP Color LaserJet Managed Flow MFP E786z,Italy,Jan,Cannot be grouped
1,PHITTA-PM0010,1476.0,Torre Annunziata,LIBERAZIONE-DAL-MALIGNO-Preghiere (1).pdf,27-Jan-25,CZBBS5V01S,Torre Annunziata,ITTA,HP Color LaserJet Enterprise Flow MFP X677z,Italy,Jan,Cannot be grouped
2,GXINKL-PM00006,1326.0,Kalwe (Technical Operations) (Sandoz),Week_1-52_SFG_2024.xlsx,6-Feb-25,CNNKL4R5PK,INKL,INKL,HP LaserJet Enterprise Managed MFP M527dnm,India,Feb,Cannot be grouped
3,PHUSMH-PM7004,1.0,Millburn,Microsoft Word - SPC-WIP-121924,2-Jan-25,CN215FY0TR,Millburn - NJ 07041,USMH,HP PageWide Managed Color Flow MFP E58650z,USA,Jan,Cannot be grouped
4,PHUSMH-PM7004,1.0,Millburn,Microsoft Word - SPC-WIP-121924,2-Jan-25,CN215FY0TR,Millburn - NJ 07041,USMH,HP PageWide Managed Color Flow MFP E58650z,USA,Jan,Cannot be grouped


In [31]:
df_unlabeled.to_csv("predicted labels.csv", index=False)

In [41]:
from sklearn.model_selection import train_test_split

# Assuming df_train contains your 10K with 'Cluster name'
X_full = model.encode(df_train['JobName'].astype(str).tolist(), show_progress_bar=True)
y_full = le.fit_transform(df_train['Cluster name'])

X_train, X_val, y_train, y_val = train_test_split(X_full, y_full, test_size=0.2, random_state=42)


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [42]:
from xgboost import XGBClassifier

clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
clf.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:22:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [43]:
y_val_pred = clf.predict(X_val)


In [44]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Overall accuracy
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=le.classes_))


Validation Accuracy: 0.9840

Classification Report:
                                              precision    recall  f1-score   support

                          ASUN excel reports       1.00      1.00      1.00        14
                           Cannot be grouped       0.98      0.99      0.98      1041
                 Chrome extension html files       0.83      1.00      0.91        15
                             Crystal Reports       1.00      0.95      0.98        22
                 Date cluster jobs and snagx       1.00      0.94      0.97        16
                            Document cluster       1.00      1.00      1.00         7
                                   Email Job       1.00      1.00      1.00        68
                 FRM 8  and 7 series reports       1.00      0.94      0.97        35
                            FRM 8 series pdf       0.95      0.97      0.96        62
                            FRM excel report       1.00      1.00      1.00         6
 

In [45]:
# Predict class probabilities instead of labels
y_val_proba = clf.predict_proba(X_val)

# Example: Confidence of first 5 predictions
import numpy as np

for i in range(5):
    top_confidence = np.max(y_val_proba[i])
    predicted_class = le.inverse_transform([y_val_pred[i]])[0]
    print(f"Predicted: {predicted_class}, Confidence: {top_confidence:.2f}")


Predicted: Microsoft word ZININSP document, Confidence: 1.00
Predicted: Unattended print, Confidence: 1.00
Predicted: Cannot be grouped, Confidence: 1.00
Predicted: Cannot be grouped, Confidence: 1.00
Predicted: Cannot be grouped, Confidence: 1.00
